# Signal Synchronization


In [ ]:
import warnings

import numpy as np
try:
    import cupy as cp
    NO_CUPY = False
except ImportError:
    warnings.warn("CuPy is not installed (needed for GPU acceleration). Falling back to CPU, which might be extremly slow!")
    NO_CUPY = True
from scipy import signal
import matplotlib.pyplot as plt

from datetime import datetime

import sys
sys.path.append('../')
import rtl_sdr_pr.ioutils
import rtl_sdr_pr.processing

## FIR-based sub-sample Synchronization


In [ ]:
def frac_delay_fir(ntaps, u):
    if u % 1 == 0:
        u += np.finfo(float).eps
    
    N = ntaps - 1
    n = np.linspace(-N/2, N/2, ntaps)
    win = signal.chebwin(ntaps, 70)
    return np.multiply(np.sinc(n - u), win.T)

In [ ]:
win = np.hanning(7)/4
x = np.convolve(win, np.ones(20))
b_zero = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
y1 = np.convolve(b_zero, x)

ntaps = 19
b_dly = frac_delay_fir(ntaps, 0.5)
b_adv = frac_delay_fir(ntaps, -0.5)
y2 = np.convolve(b_dly, x)
y3 = np.convolve(b_adv, x)

plt.figure()
plt.plot(x)
plt.grid(True)

plt.figure()
plt.plot(y1)
plt.grid(True)

plt.figure()
plt.plot(y1)
plt.plot(y2)
plt.plot(y3)
plt.grid(True)

plt.figure()
plt.plot(b_dly)
plt.plot(b_adv)
plt.grid(True)

## Correlation-based Synchronization

Use timestamp information (if available) and cross-correlation to find sample offset in recordings. This approach assumes the receiver hardware is phase and frequency coherent (i.e. sharing a common local oscillator).

First some setup-work, which includes reading file headers and determining a rough offset based on recording timestamps.


In [ ]:
CPI = 0.2 # in seconds

ref_file_path = "../data/pluto_a_ref.2021-07-27T15_24_17_598.sdriq"
surv_file_path = "../data/pluto_b_surv.2021-07-27T15_24_21_819.sdriq"

_, ref_hdr = rtl_sdr_pr.ioutils.read_sdriq_samples(ref_file_path, 0, 0)
_, surv_hdr = rtl_sdr_pr.ioutils.read_sdriq_samples(surv_file_path, 0, 0)

assert ref_hdr["sample_rate"] == surv_hdr["sample_rate"]
assert ref_hdr["center_frequency"] == surv_hdr["center_frequency"]

sample_rate = ref_hdr["sample_rate"]
center_frequency = ref_hdr['center_frequency']
num_samples_in_cpi = CPI * sample_rate

print(f"Sample rate: {sample_rate / 1e6:0.1f} MHz, Center frequency: {center_frequency / 1e6:0.1f} MHz")
print(f"Start of reference channel: {ref_hdr['start_time_stamp']} -> {datetime.fromtimestamp(ref_hdr['start_time_stamp'])}")
print(f"Start of surveillance channel: {surv_hdr['start_time_stamp']} -> {datetime.fromtimestamp(surv_hdr['start_time_stamp'])}")

time_diff = surv_hdr['start_time_stamp'] - ref_hdr['start_time_stamp']
estim_sample_shift = time_diff * sample_rate

print(f"Time delta: {time_diff} sec, Estimated sample shift: {estim_sample_shift}")

ref_samples, _ = rtl_sdr_pr.ioutils.read_sdriq_samples(ref_file_path, int(num_samples_in_cpi * 2), max(0, int(estim_sample_shift - num_samples_in_cpi)))
surv_samples, _ = rtl_sdr_pr.ioutils.read_sdriq_samples(surv_file_path, int(num_samples_in_cpi * 2), max(0, -int(estim_sample_shift - num_samples_in_cpi)))

sub_sample_factor = 1
ref_samples = ref_samples[::sub_sample_factor]
surv_samples = surv_samples[::sub_sample_factor]

Now correlate and find the correlation peak.


In [ ]:
if NO_CUPY:
    # PLEASE INSTALL CUPY! OTHERWISE THIS IS SOOO INCREADIBLY SLOW!
    corr = np.correlate(ref_samples, surv_samples, mode="same")
else:
    corr = cp.correlate(cp.asarray(ref_samples), cp.asarray(surv_samples), mode="same").get()
corr_mag = np.abs(corr)
corr_mag_db = 10 * np.log10(corr_mag / np.max(corr_mag))

peak_idx = np.argmax(corr_mag)
peak_time_offset = (peak_idx - corr_mag.size / 2) / sample_rate * sub_sample_factor

print(f"Correlation peak at {peak_idx} with time offset of {peak_time_offset * 1000:0.3f} msec after initial estimation")

In [ ]:
plt.figure(figsize=(40, 7))
plt.plot(np.linspace(-corr_mag_db.size / 2 / sample_rate * 1000 * sub_sample_factor, corr_mag_db.size / 2 / sample_rate * 1000 * sub_sample_factor, corr_mag_db.size), corr_mag_db)
plt.xlabel("time delta [msec]")
plt.ylabel("correlation [dB]")
plt.xticks(np.arange(-CPI * 1000, CPI * 1000 + 1, step=100, dtype=np.int32))
plt.grid()

plt.annotate(f"Peak\n"
             f"sample offset: {peak_idx * sub_sample_factor}\n"
             f"time offset: {peak_time_offset * 1000:0.3f} msec",
             xy=(peak_time_offset * 1000, corr_mag_db[peak_idx]),
             xytext=(-100, 30),
             xycoords="data",
             textcoords="offset pixels",
             arrowprops={'arrowstyle': 'wedge'});

## PSS Synchronization

5G Broadcast communication uses two types of signals to allow a UE to synchronize with the eNodeB. The Primary Synchronization Signal (PSS) is one of three Zadoff-Chu sequences given by

$$
  d_u(n) = \begin{cases}
             \mathrm{exp}\left(-\mathrm{j} \frac{\pi u n (n + 1)}{63}\right) & n = 0, 1, \dots, 30 \\
             \mathrm{exp}\left(-\mathrm{j} \frac{\pi u n (n + 1)(n + 2)}{63}\right) & n = 31, 32,\dots, 61 \\
           \end{cases}
$$

where the Zadoff-Chu root sequence index $u$ is given by the following table

|Physical Layer Identity $N^{(2)}_\mathrm{ID}$|Root Index $u$|
|---------------------|--------------|
|$0$|$25$|
|$1$|$29$|
|$2$|$34$|

We start by generating the Zadoff-Chu sequences mentioned above.


In [ ]:
n, u = np.meshgrid(np.arange(0, 62), np.array([25, 29, 34]))
z = np.zeros_like(n, dtype=np.complex128)
z[:, :31] = np.exp(-1j * (np.pi * u[:, :31] * n[:, :31] * (n[:, :31] + 1)) / 63)
z[:, 31:] = np.exp(-1j * (np.pi * u[:, 31:] * (n[:, 31:] + 1) * (n[:, 31:] + 2)) / 63)

Test the generated sequences by calculating their auto-correlation. A neat property of Zadoff-Chu sequences is that their auto-correlation is $\approx 0$ for all non-zero lag terms.


In [ ]:
print(f"First 5 elements of each Zadoff-Chu sequence\n{z[:, :5]}")

z_gpu = cp.asarray(z)
corr_abs = cp.abs(cp.correlate(z_gpu[0, :], z_gpu[0, :], mode='full')).get()

plt.figure(figsize=(15, 5))
plt.plot(np.arange(-corr_abs.shape[0] // 2 + 1, corr_abs.shape[0] // 2 + 1), corr_abs / np.max(corr_abs))
plt.xlabel("lag [symbols]")
plt.ylabel("norm. correlation")
plt.title("auto-correlation of first Zadoff-Chu sequence ($u = 25$)")
plt.grid()

In [ ]:
freqz = cp.abs(cp.fft.fftshift(cp.fft.fft(cp.asarray(surv_samples[:4096])))).get()
signal_bw = 5e6

plt.figure(figsize=(20, 5))
plt.plot(np.linspace((center_frequency - sample_rate / 2) * 1e-6, (center_frequency +  sample_rate / 2) * 1e-6, 4096), freqz)
plt.title("Frequency Spectrum of Observation Signal")
plt.xlabel("Frequency [MHz]")
plt.ylabel("Amplitude")
plt.grid()
plt.axvspan((center_frequency - signal_bw / 2) * 1e-6, (center_frequency + signal_bw / 2) * 1e-6, color="red", alpha=0.2);

### PSS Reference Subframe Generation

Derive modulation parameters based on some constants.


In [ ]:
num_dl_rb = int(0.9 * signal_bw / 180e3)
sc_per_rb = 12
num_dl_sc = num_dl_rb * sc_per_rb
print(f"Assuming {num_dl_rb:d} downlink resource-blocks given channel bandwidth of {signal_bw * 1e-6:.1f} MHz")

num_fft_bins = int(2**np.ceil(np.log2(num_dl_sc))) # find closest power of 2 fitting all sub-carriers
first_sc_idx = num_fft_bins // 2 - num_dl_sc // 2 - 1 # offset of first sub-carrier in larger-than-necessary set of FFT-bins

cyc_pref = np.tile(np.array([160, 144, 144, 144, 144, 144, 144]), 2)
# 2x + 12*0.9x = 2048
# x(2 + 12*0.9) = 2048
# x = 2048 / (2 + 12*0.9) = 160
cyc_pref = (cyc_pref / np.sum(cyc_pref) * num_fft_bins).astype(int)

symbols_per_slot = 7
slots_per_subframe = 2
symbols_per_subframe = symbols_per_slot * slots_per_subframe
samples_per_subframe = int(num_fft_bins * symbols_per_slot * slots_per_subframe + np.sum(cyc_pref))

num_pss_rb = 5 # PSS is always located in the middle 5 RBs
pss_symbol_idx_in_slot = 6 # assuming 0,1,...,6 symbols per slot
pss_sc_offset_in_symbol = int(((num_dl_rb - num_pss_rb) / 2) * sc_per_rb) # sub-carrier offset within symbol #6 that points to the start of PSS

We will now construct a reference subframe containing only the PSS sequence located in the middle 5 RBs.


In [ ]:
grid = np.zeros((z.shape[0], num_dl_sc, symbols_per_subframe), dtype=np.complex128)

grid[:, np.arange(pss_sc_offset_in_symbol, pss_sc_offset_in_symbol + z.shape[1]), pss_symbol_idx_in_slot] = z[:, :]

ifft_in = np.zeros((z.shape[0], num_fft_bins, symbols_per_subframe), dtype=np.complex128)
ifft_in[:, first_sc_idx : first_sc_idx+num_dl_sc // 2 + 1, :] = grid[:, np.arange(num_dl_sc // 2 + 1), :]
ifft_in[:, first_sc_idx + 1 + (num_dl_sc // 2 + 1) : first_sc_idx + num_dl_sc + 1, :] = grid[:, np.arange(num_dl_sc // 2 + 1, num_dl_sc), :]

ifft_out = cp.fft.ifft(cp.fft.fftshift(cp.asarray(ifft_in), axes=1), axis=1)

pss_waveforms = cp.empty((z.shape[0], samples_per_subframe), dtype=np.complex128)

for sym_idx in np.arange(symbols_per_subframe):
    pss_waveforms[:, sym_idx * num_fft_bins + np.sum(cyc_pref[:sym_idx]) : sym_idx * num_fft_bins + np.sum(cyc_pref[:sym_idx + 1])] = ifft_out[:, -cyc_pref[sym_idx]:, sym_idx] # copy cyclic prefix to begining of each time-slot
    pss_waveforms[:, sym_idx * num_fft_bins + np.sum(cyc_pref[:sym_idx + 1]) : (sym_idx + 1) * num_fft_bins + np.sum(cyc_pref[:sym_idx + 1])] = ifft_out[:, :, sym_idx] # fill rest of time-slot with OFDM symbol


Our cell search works by generating a subframe containing only the PSS or SSS (first PSS then in a second run SSS). These reference subframes are then sequentially cross-correlated across the input channel data, marking the PSS/SSS (aka cell-id) combinations with the highest peaks. After going through PSS/SSS combinations, the one with the highest peak is determined the correct cell-id.
In the block above we generated a grid of resource blocks matching the input data. The grid is filled with either a Zadoff-Chu- (PSS) or m-sequence (SSS) in frequency domain. To make the correlation, the data must be OFDM modulated into a time-domain signal. Like any practical OFDM implementation LTE uses a cyclic prefix (to alleviate issues with multi-path reception). That is, each data point is converted into an OFDM symbol by means of Inverse Fourier Transform (IFT), then a specific amount of samples from the end of each symbol (after IFT!) are copied to the front of each symbol respectively. The quantity of copied samples depends on the position of the symbol inside the slot as well as channel settings (normal or extended cyclic prefix, sub-carrier spacing, etc.). For normal cyclic prefix the sum of all CPs in a subframe is 2048 for a 20MHz channel.

$$
N_{\text{CP}, l} = \begin{cases}
160 & \mathrm{mod}(l, 7) = 0 \\
144 & \mathrm{mod}(l, 7) = 1, 2, \dots, 6 \\
\end{cases}
$$

For smaller channels this calculation is scalled down in steps of powers of two (1024, 512, 256, ...), which ensures that  is always an integer number of samples. Like in the case of 5MHz, 25 RBs equating to 300 sub-carriers (12 SC per 1 RB) are available. The closest power of two after 300 would be 512. The 300 SCs are copied to the center IFT bins, padding the rest with zeros. This now virtually 512 SC wide symbol results in a larger required sample count per slot -> subframe -> frame and finally samples per second.

Numer of samples prefixed per slot:
$$
N_{\text{CP}, \text{slot}} = 512
$$
Number of samples per symbol, i.e. size of FFT:
$$
N_{\text{sym}} = 512
$$
Resulting number of samples per second:
$$
N = \left( N_{\text{CP}, \mathrm{slot}} + \underset{\text{symbols per subframe}}{14} \cdot N_{\mathrm{sym}} \right) \cdot \underset{\text{subframes per frame}}{10} \cdot \underset{\text{frames per sec}}{100} = 7.68 \cdot 10^6
$$

Thus modulation occurs at a much higher sampling rate of 7.68MHz than necessary.
Now the sample rate of our recording might be lower than that (because the physical channel may only be 5MHz wide), but we have to artificially widen it to match the 7.68MHz of our reference subframe. This is done here, by simply padding with zeros in frequency domain.


In [ ]:
T_S = 1 / (15000 * 2048)
T_FRAME = T_S * 307200

subframes_per_frame = 10
frames_per_second = 100
gen_sample_rate = samples_per_subframe * subframes_per_frame * frames_per_second
frame_count = 20

sample_rate_diff = gen_sample_rate - sample_rate

obsrv_samples = cp.asarray(surv_samples[:int(sample_rate * T_FRAME * frame_count)])

obsrv_freqs = cp.fft.fftshift(cp.fft.fft(obsrv_samples))
padded_obsrv_fftout = np.pad(obsrv_freqs, pad_width=(int(sample_rate_diff * T_FRAME * frame_count) // 2, int(sample_rate_diff * T_FRAME * frame_count) // 2))
upsampled_obsrv_samples = cp.fft.ifft(cp.fft.fftshift(padded_obsrv_fftout))

plt.figure(figsize=(20,7))
plt.plot(np.linspace(-gen_sample_rate / 2, gen_sample_rate / 2, padded_obsrv_fftout.shape[0]) * 1e-6, cp.abs(padded_obsrv_fftout).get())
plt.title("Frequency Spectrum of (padded) Observation Signal")
plt.xlabel("Frequency [MHz]")
plt.ylabel("Amplitude")
closest_500_khz_divisor = (gen_sample_rate / 2 - np.mod(gen_sample_rate / 2, 5e5**np.fix(np.log(gen_sample_rate / 2) / np.log(5e5)))) * 1e-6
plt.xticks(np.hstack([np.arange(-closest_500_khz_divisor, closest_500_khz_divisor + 0.1, step=0.5), np.array([-gen_sample_rate / 2 * 1e-6, gen_sample_rate / 2 * 1e-6])]))
plt.grid()

In [ ]:
corr_mags = cp.vstack([ cp.abs(cp.correlate(upsampled_obsrv_samples, pss_waveform, mode="valid")) for pss_waveform in pss_waveforms ])
x = np.arange(corr_mags.shape[1]) * 1000 / gen_sample_rate

peak_idx = cp.argmax(corr_mags, axis=1).get()
peak_time_offset = peak_idx / gen_sample_rate

print(f"Correlation peaks at {np.array2string(peak_idx)} with time offset of {np.array2string(peak_time_offset * 1000, precision=3)} msec")

In [ ]:
for idx, corr_mag in enumerate(corr_mags[:]):
    corr_db = 10 * np.log10(corr_mag / cp.max(corr_mag))
    plt.figure(figsize=(35, 3))
    plt.plot(x, corr_db.get())
    plt.title(f"Correlation of Generated PSS {idx} with Observation Signal")
    plt.xlabel("time delta [msec]")
    plt.ylabel("correlation [dB]")
    plt.xticks(np.linspace(0, T_FRAME * frame_count * 1000, frame_count * 2 + 1))
    plt.grid()

    plt.annotate(f"Peak\n"
                f"sample offset: {peak_idx[idx]}\n"
                f"time offset: {peak_time_offset[idx] * 1000:0.3f} msec",
                xy=(peak_time_offset[idx] * 1000, corr_db[peak_idx[idx]]),
                xytext=(50, 40),
                xycoords="data",
                textcoords="offset pixels",
                arrowprops={'arrowstyle': 'wedge'});

    if peak_time_offset[idx] + T_FRAME / 2 < frame_count * T_FRAME:
        plt.axvspan((peak_time_offset[idx] + T_FRAME * 4) * 1000 - 0.75, (peak_time_offset[idx] + T_FRAME * 4) * 1000 + 0.75, color='red', alpha=0.4)
        plt.annotate(f"Estimated next PSS\n"
                    f"time offset: {(peak_time_offset[idx] + T_FRAME * 4) * 1000:0.3f} msec",
                    xy=((peak_time_offset[idx] + T_FRAME * 4) * 1000, 0),
                    xytext=(-100, 40),
                    xycoords="data",
                    textcoords="offset pixels",
                    arrowprops={'arrowstyle': 'wedge'});

    if peak_time_offset[idx] - T_FRAME / 2 > 0:
        plt.axvspan((peak_time_offset[idx] - T_FRAME * 4) * 1000 - 0.75, (peak_time_offset[idx] - T_FRAME * 4) * 1000 + 0.75, color='red', alpha=0.4)
        plt.annotate(f"Estimated previous PSS\n"
                    f"time offset: {(peak_time_offset[idx] - T_FRAME * 4) * 1000:0.3f} msec",
                    xy=((peak_time_offset[idx] - T_FRAME * 4) * 1000, 0),
                    xytext=(-100, 40),
                    xycoords="data",
                    textcoords="offset pixels",
                    arrowprops={'arrowstyle': 'wedge'});
